In [1]:
%matplotlib inline

import librosa
import matplotlib.pyplot as plt
import librosa.display
import numpy as np
import torch
import os
import sys
import glob
import pickle
import copy
import random
import time
import traceback
import shutil

from multiprocessing import Process, Manager

# Config

In [2]:
PROJECT_ROOT = '/workspace/GREAT_ASV_system/'
sys.path.append(PROJECT_ROOT)

In [3]:
OPT_INDEX = '/workspace/DATASET/server9/STD_VOX_EGS'
if not os.path.exists(OPT_INDEX):
    os.mkdir(OPT_INDEX)

In [4]:
VOX_TRAIN_DIR = '/workspace/DATASET/server9/voxceleb'
AUG_INFO_DIR = '/workspace/DATASET/server9/STD_musan&rir_info'

In [5]:
INFO_DIR = os.path.join(OPT_INDEX, 'VOX2_INFO')
if not os.path.exists(INFO_DIR):
    os.mkdir(INFO_DIR)

spk2id = os.path.join(INFO_DIR, 'spk2id')
spk2utt_dict = os.path.join(INFO_DIR, 'spk2utt_dict')
spk2utt_train_dict = os.path.join(INFO_DIR, 'spk2utt_train_dict')
spk2utt_val_dict = os.path.join(INFO_DIR, 'spk2utt_val_dict')
spk2utt_train_len = os.path.join(INFO_DIR, 'spk2utt_train_len')
spk2utt_val_len = os.path.join(INFO_DIR, 'spk2utt_val_len')

In [6]:
val_utt_num = 200

# Make spk2utt dict

In [7]:
spk2utt = {}
spk2utt_train = {}
spk2utt_val = {}

with open(spk2id, 'w') as f:
    pass

label_dir = glob.glob(VOX_TRAIN_DIR+'/vox2/dev/aac/*')
label = [i.split('/')[-1] for i in label_dir]

for count, i in enumerate(label):
    with open(spk2id, 'a') as f:
        f.write(str(count)+','+i+'\n')
    spk2utt[count] = glob.glob(os.path.join(VOX_TRAIN_DIR+'/vox2/dev/aac/', i)+'/*/*.wav')

In [8]:
len(spk2utt)

5994

In [9]:
count = 0
count_list = []
for i in spk2utt:
    count += len(spk2utt[i])
    count_list.append(len(spk2utt[i]))
print(count)

1092009


In [10]:
with open(spk2utt_dict, 'wb') as handle:
    pickle.dump(spk2utt, handle)

with open(spk2utt_dict, 'rb') as handle:
    spk2utt = pickle.load(handle)

In [11]:
utt_length = []
for i in spk2utt:
    utt_length.append(len(spk2utt[i]))

In [12]:
utt_length = np.argsort(utt_length)[::-1]

In [13]:
spk2utt_train = copy.deepcopy(spk2utt)
for i in range(val_utt_num):   
    spk2utt_val[utt_length[i]] = [spk2utt[utt_length[i]][0]]
    spk2utt_train[utt_length[i]] = spk2utt[utt_length[i]][1:]

In [14]:
count = 0
count_list = []
for i in spk2utt_train:
    count += len(spk2utt_train[i])
    count_list.append(len(spk2utt[i]))
print(count)

1091809


In [15]:
with open(spk2utt_train_dict, 'wb') as handle:
    pickle.dump(spk2utt_train, handle)

In [16]:
count = 0
count_list = []
for i in spk2utt_val:
    count += len(spk2utt_val[i])
    count_list.append(len(spk2utt[i]))
print(count)

200


In [17]:
with open(spk2utt_val_dict, 'wb') as handle:
    pickle.dump(spk2utt_val, handle)

## Use multi-process to check read wav

In [18]:
manager = Manager()
spk2utt_train_len_dict_m = manager.list()
eer_m = manager.list()

In [19]:
# err = []
with open(spk2utt_train_dict, 'rb') as handle:
    spk2utt_train = pickle.load(handle)

In [20]:
spk2utt_train_splitlist = []
num_per_process = len(spk2utt_train) // 10
for i in range(9):
    spk2utt_train_splitlist.append([j for j in range(i*num_per_process, (i+1)*num_per_process)])
    spk2utt_train_len_dict_m.append({})
spk2utt_train_splitlist.append([j for j in range(9*num_per_process, len(spk2utt_train))])
spk2utt_train_len_dict_m.append({})

In [21]:
def read_wav_m(p, spk2utt_train, spk2utt_train_split, spk2utt_train_len_dict_m, eer_m):
    spk2utt_train_len_dict = {}
    for count, i in enumerate(spk2utt_train_split):
        for utt in spk2utt_train[i]:
            try:
                spk2utt_train_len_dict[utt] = librosa.core.load(utt, sr=16000)[0].shape[0]
            except Exception:
                eer_m.append([p, utt])
                continue
        print(count)
    spk2utt_train_len_dict_m[p] = spk2utt_train_len_dict

In [22]:
with open(spk2utt_train_dict, 'rb') as handle:
    spk2utt_train = pickle.load(handle)

processes = [Process(target = read_wav_m, args = (i, spk2utt_train, spk2utt_train_splitlist[i], spk2utt_train_len_dict_m, eer_m)) for i in range(10)]
[p.start() for p in processes]
joined = [p.join() for p in processes]

0
01

0
1
0
1
00

0
1
1
2
1
1
2
2
3
2
3
4
3
4
5
3
4
2
4
0
6
1
7
3
5
5
2
5
3
4
6
0
2
4
6
5
6
5
0
7
2
1
6
1
2
3
7
6
7
3
8
8
7
3
4
8
8
8
4
9
9
2
9
3
9
4
5
10
5
6
9
11
10
10
7
11
7
11
12
8
6
12
10
12
4
7
8
5
13
11
9
13
5
8
10
6
12
9
9
10
14
6
10
11
11
12
13
14
10
13
14
11
7
13
12
12
7
15
16
13
14
17
15
14
13
15
11
14
8
15
18
15
16
9
14
16
16
17
12
15
17
17
15
18
16
19
10
11
18
19
20
17
12
18
1321

13
16
22
18
14
19
20
15
16
17
16
21
18
20
19
17
18
21
23
24
25
8
20
22
17
21
22
23
26
27
28
19
20
21
18
19
20
21
23
22
23
22
24
19
22
24
20
21
23
19
25
9
10
11
29
12
14
20
15
30
21
24
22
23
24
25
25
31
26
25
24
25
16
17
18
32
13
23
26
27
26
22
33
14
24
27
15
27
34
25
16
26
28
28
35
28
23
36
19
24
26
26
20
27
27
25
27
28
21
29
26
28
27
30
28
28
29
29
17
22
31
29
30
29
18
32
30
30
31
29
31
19
33
30
30
32
34
31
29
32
35
31
32
36
33
37
30
31
32
33
23
33
37
33
34
34
34
20
35
32
38
36
21
35
37
33
36
38
34
34
37
31
24
38
32
35
35
38
33
36
36
25
39
39
34
26
40
35
39
37
35
36
41
37
38
38
39
22
36
39
23
27

239
249
246
226
250
239
251
240
252
227
245
253
221
198
246
254
247
199
222
255
223
228
248
224
256
212
229
257
248
225
240
258
230
226
259
241
213
231
214
249
215
232
216
241
200
227
260
250
233
261
201
251
262
247
242
202
243
217
248
252
242
228
234
229
244
230
235
245
243
246
263
249
244
249
264
250
245
253
247
218
236
231
248
246
265
249
250
219
237
254
247
220
266
203
251
250
255
267
248
251
238
252
251
239
249
256
268
252
269
252
221
257
222
270
223
253
253
271
204
232
205
233
250
224
254
240
234
272
253
255
254
256
254
273
255
206
274
275
258
241
256
255
207
259
225
257
256
242
260
258
276
226
235
257
259
277
261
227
278
228
251
279
260
262
263
258
280
261
259
281
208
264
236
282
237
229
257
238
258
283
252
262
259
239
263
260
243
284
244
230
245
285
253
246
265
260
286
264
247
261
254
209
231
210
261
255
248
262
265
256
240
249
241
263
257
242
232
250
233
287
266
266
243
234
244
264
262
235
258
263
288
265
245
267
264
289
265
290
246
267
266
236
247
291
237
292
268
268
293
259


450
461
466
450
375
428
451
425
462
426
452
376
463
453
453
377
464
435
454
454
378
451
436
455
429
428
379
465
430
429
427
431
452
430
380
432
467
466
455
433
468
456
428
429
381
456
457
430
434
431
435
467
453
437
457
458
436
438
432
469
433
382
454
434
435
468
439
458
470
437
471
383
384
459
472
460
440
441
459
442
436
438
385
460
455
437
431
443
461
439
438
473
456
462
386
432
444
463
461
464
474
475
462
387
440
388
476
463
433
477
445
389
457
458
465
439
459
390
460
440
434
464
478
391
466
479
392
465
435
446
469
441
393
466
442
436
470
480
443
471
467
394
481
461
472
437
462
468
463
444
441
482
438
447
439
467
483
468
395
484
396
485
464
469
397
445
465
486
446
469
440
473
441
398
447
470
399
470
471
487
400
448
442
442
448
449
472
443
449
443
444
471
473
466
401
445
402
403
444
472
467
450
446
474
445
450
451
475
468
452
469
453
447
473
446
470
447
454
448
451
476
452
404
477
448
478
474
488
475
449
453
405
471
489
472
454
455
406
450
455
456
473
456
451
490
491
452
407
479
480


In [23]:
with open(OPT_INDEX+'/spk2utt_train_len_TMP', 'wb') as handle:
    pickle.dump(spk2utt_train_len_dict_m, handle)

In [24]:
spk2utt_train_len_dict = {}
for this_dict in spk2utt_train_len_dict_m:
    for count, i in enumerate(this_dict):
        spk2utt_train_len_dict[i] = this_dict[i]
#         print(count)
print('train_len_num', len(spk2utt_train_len_dict))

train_len_num 1091809


In [25]:
err_list = list(eer_m)

In [26]:
err_list

[]

In [27]:
with open(spk2utt_train_len, 'wb') as handle:
    pickle.dump(spk2utt_train_len_dict, handle)

In [ ]:
# spk2utt_train_len_dict = {}

In [ ]:
# err = []
# with open(spk2utt_train_dict, 'rb') as handle:
#     spk2utt_train = pickle.load(handle)

# for i in spk2utt_train:
#     for utt in spk2utt_train[i]:
#         try:
#             spk2utt_train_len_dict[utt] = librosa.core.load(utt, sr=16000)[0].shape[0]
#         except Exception:
#             err.append(utt)
#             continue
#     print(i)
            
# with open(spk2utt_train_len, 'wb') as handle:
#     pickle.dump(spk2utt_train_len_dict, handle)

In [28]:
spk2utt_val_len_dict = {}

In [29]:
err = []
with open(spk2utt_val_dict, 'rb') as handle:
    spk2utt_val = pickle.load(handle)

for i in spk2utt_val:
    for utt in spk2utt_val[i]:
        try:
            spk2utt_val_len_dict[utt] = librosa.core.load(utt, sr=16000)[0].shape[0]
        except Exception:
            err.append(utt)
            continue
    print(i)

with open(spk2utt_val_len, 'wb') as handle:
    pickle.dump(spk2utt_val_len_dict, handle)

4173
4217
492
4307
2008
4312
1979
1977
1973
4339
4343
1968
509
511
517
4369
5522
491
488
2023
5569
464
2079
2075
2072
467
4263
4268
2028
2057
2055
478
2045
4286
5553
1936
1935
4377
1696
4500
571
4502
4515
4521
4529
4544
4496
1688
1681
4557
4558
1673
1671
1744
559
4383
1837
4392
1903
5511
1888
536
1877
1831
1768
1823
1813
1807
4456
547
1795
2089
4212
4584
2130
4014
2361
4021
359
4022
5663
2348
2347
4031
2339
2337
4049
4052
2324
2320
354
4013
4011
2418
3964
2421
3966
5695
3969
5693
3975
5674
3981
2396
2370
4009
2367
5675
372
2302
2300
5599
4142
4147
2192
5605
4160
4164
2180
415
5200
2170
4181
450
4195
2147
5617
5619
385
2241
2297
4092
390
4097
4113
4114
4122
2211
5628
4125
2228
2218
2216
411
1657
4585
2430
4908
4998
810
5000
5001
1138
5004
5010
5013
1121
1120
1118
5030
5031
820
1109
5290
4994
4992
1198
770
4913
4915
1221
1216
1200
784
1152
4951
4959
1175
1157
798
4977
1104
1096
5265
905
979
5152
894
5154
966
5158
5213
5138
959
939
931
921
918
920
989


In [30]:
err

[]

# Make train data

In [32]:
from preprocessing.Utils.preprocessing_3type import ThreeTypes_IterableDataset

In [33]:
data_dir_dict = {}

data_dir_dict['spk2utt_train_dict'] = os.path.join(INFO_DIR, 'spk2utt_train_dict')
data_dir_dict['music_dict'] = os.path.join(AUG_INFO_DIR, 'music_dict')
data_dir_dict['noise_dict'] = os.path.join(AUG_INFO_DIR, 'noise_dict')
data_dir_dict['babble_dict'] = os.path.join(AUG_INFO_DIR, 'babble_dict')
data_dir_dict['rir_dict'] = os.path.join(AUG_INFO_DIR, 'rir_dict')

In [34]:
data_len_dict = {}

data_len_dict['spk2utt_train_len'] = os.path.join(INFO_DIR, 'spk2utt_train_len')
data_len_dict['music_len'] = os.path.join(AUG_INFO_DIR, 'music_len')
data_len_dict['noise_len'] = os.path.join(AUG_INFO_DIR, 'noise_len')
data_len_dict['babble_len'] = os.path.join(AUG_INFO_DIR, 'babble_len')

## Multi episode process

In [35]:
PROCESS = 1500
RUNNING_P = 10

MFCC_O = os.path.join(OPT_INDEX, 'TRAIN_VOX2_MFCC')
LOGMELFB_O = os.path.join(OPT_INDEX, 'TRAIN_VOX2_FB')
RAWWAV_O = os.path.join(OPT_INDEX, 'TRAIN_VOX2_RAW')
EER_LOG = os.path.join(OPT_INDEX, 'EER_LOG')
G_LOG = os.path.join(OPT_INDEX, 'G_LOG')

In [36]:
if not os.path.exists(MFCC_O):
    os.mkdir(MFCC_O)
else:
    shutil.rmtree(MFCC_O)
    
if not os.path.exists(LOGMELFB_O):
    os.mkdir(LOGMELFB_O)
else:
    shutil.rmtree(LOGMELFB_O)
    
if not os.path.exists(RAWWAV_O):
    os.mkdir(RAWWAV_O)
else:
    shutil.rmtree(RAWWAV_O)

with open(EER_LOG, 'w') as f:
    pass

with open(G_LOG, 'w') as f:
    pass

In [37]:
config = {}

config['sr'] = 16000
config['repeats'] = 1
config['batch_size'] = 1

In [38]:
def iter_data_preload(dataset, i):
    dataset.get_random_list()
    try:
        start_time = time.time()
        for count, (rw, fb, mfcc, label) in enumerate(dataset):
            file_name = str(i)+str('_')+str(count)
            with open(os.path.join(RAWWAV_O, file_name), 'wb') as handle:
                pickle.dump((rw.astype(np.float16), label.astype(np.int16)), handle)
                
            with open(os.path.join(LOGMELFB_O, file_name), 'wb') as handle:
                pickle.dump((fb.astype(np.float16), label.astype(np.int16)), handle)
                
            with open(os.path.join(MFCC_O, file_name), 'wb') as handle:
                pickle.dump((mfcc.astype(np.float16), label.astype(np.int16)), handle)
            
            with open(G_LOG, 'a') as f:
                f.write(str(time.time()-start_time)+'\n')
            start_time = time.time()
            
    except Exception:
        with open(EER_LOG, 'a') as f:
            traceback.print_exc(file=f)
        raise Exception

dataset = ThreeTypes_IterableDataset(config, data_dir_dict, data_len_dict)
dataset.noise_data_preload()

p_list = []
for i in range(PROCESS // RUNNING_P):
    p_list.append([j for j in range(i*RUNNING_P, i*RUNNING_P+RUNNING_P)])
if (PROCESS % RUNNING_P) != 0:
    p_list.append([j for j in range((i+1)*RUNNING_P, PROCESS)])

print('episode', len(p_list))
print('last episode', p_list[-1])

start_time = time.time()
for ps in p_list:
    print(ps[0])
    processes = [Process(target = iter_data_preload, args = (dataset, i)) for i in ps]
    [p.start() for p in processes]
    joined = [p.join() for p in processes]

preloading music_dict
100
200
300
400
500
600
preloading noise_dict
100
200
300
400
500
600
700
800
900
preloading babble_dict
100
200
300
400
episode 150
last episode [1490, 1491, 1492, 1493, 1494, 1495, 1496, 1497, 1498, 1499]
0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810
820
830
840
850
860
870
880
890
900
910
920
930
940
950
960
970
980
990
1000
1010
1020
1030
1040
1050
1060
1070
1080
1090
1100
1110
1120
1130
1140
1150
1160
1170
1180
1190
1200
1210
1220
1230
1240
1250
1260
1270
1280
1290
1300
1310
1320
1330
1340
1350
1360
1370
1380
1390
1400
1410
1420
1430
1440
1450
1460
1470
1480
1490


In [ ]:
# # normal
# config = {}

# config['sr'] = 16000
# config['repeats'] = 150
# config['batch_size'] = 1
# config['extended_prefectch'] = 1.0
# config['running_p'] = 10
# config['out_dir'] = '/Lun0/zhiyong/dataset/test_1/'
# config['err_log'] = '/Lun0/zhiyong/dataset/tmp_data_errlog'

# def iter_data_preload(dataset, i):
#     dataset.get_random_list()
#     try:
#         start_time = time.time()
#         for count, (data, label) in enumerate(dataset):
#             with open(config['out_dir']+str(i)+str('_')+str(count), 'wb') as handle:
#                 pickle.dump((data.astype(np.float16), label.astype(np.int16)), handle)
# #             print(time.time()-start_time)
#             with open(config['err_log'], 'a') as f:
#                 f.write(str(time.time()-start_time)+'\n')
#             start_time = time.time()
            
#             if ((count+1)%10000) == 0:
#                 print((count+1)//10000)
#     except Exception:
#         with open(config['err_log'], 'a') as f:
#             traceback.print_exc(file=f)
#         raise Exception

# dataset = VoxIterableDataset(data_dir_dict, data_len_dict, config)
# # dataset.noise_data_preload2mem()
# dataset.noise_data_preload()

# processes = [Process(target = iter_data_preload, args = (dataset, i)) for i in range(config['running_p'])]
# start_time = time.time()
# [p.start() for p in processes]
# joined = [p.join() for p in processes]
# print(time.time()-start_time)

# Make train index

In [39]:
train_mfcc_dir = os.path.join(INFO_DIR, 'train_mfcc.csv')
train_fb_dir = os.path.join(INFO_DIR, 'train_fb.csv')
train_rw_dir = os.path.join(INFO_DIR, 'train_raw.csv')

In [40]:
mfcc_len = glob.glob(MFCC_O+'/*')
fb_len = glob.glob(LOGMELFB_O+'/*')
raw_len = glob.glob(RAWWAV_O+'/*')

In [41]:
print('mfcc', len(mfcc_len))
print('fb', len(fb_len))
print('raw', len(raw_len))

mfcc 8991000
fb 8991000
raw 8991000


In [42]:
count = 0
with open(train_mfcc_dir, 'w') as f:
    for path in mfcc_len:
        assert os.path.isfile(path)
        f.write(path+'\n')
        count += 1
print('mfcc', count)

count = 0
with open(train_fb_dir, 'w') as f:
    for path in fb_len:
        assert os.path.isfile(path)
        f.write(path+'\n')
        count += 1
print('fb', count)

count = 0
with open(train_rw_dir, 'w') as f:
    for path in raw_len:
        assert os.path.isfile(path)
        f.write(path+'\n')
        count += 1
print('raw', count)

mfcc 8991000
fb 8991000
raw 8991000


In [44]:
5994 * 1500

8991000

# Make validation data

In [48]:
VAL_MFCC_O = os.path.join(OPT_INDEX, 'VAL_MFCC')
VAL_LOGMELFB_O = os.path.join(OPT_INDEX, 'VAL_FB')
VAL_RAWWAV_O = os.path.join(OPT_INDEX, 'VAL_RAW')

In [49]:
data_dir_dict = {}

data_dir_dict['spk2utt_train_dict'] = os.path.join(INFO_DIR, 'spk2utt_val_dict')
data_dir_dict['music_dict'] = os.path.join(AUG_INFO_DIR, 'music_dict')
data_dir_dict['noise_dict'] = os.path.join(AUG_INFO_DIR, 'noise_dict')
data_dir_dict['babble_dict'] = os.path.join(AUG_INFO_DIR, 'babble_dict')
data_dir_dict['rir_dict'] = os.path.join(AUG_INFO_DIR, 'rir_dict')

In [50]:
data_len_dict = {}

data_len_dict['spk2utt_train_len'] = os.path.join(INFO_DIR, 'spk2utt_val_len')
data_len_dict['music_len'] = os.path.join(AUG_INFO_DIR, 'music_len')
data_len_dict['noise_len'] = os.path.join(AUG_INFO_DIR, 'noise_len')
data_len_dict['babble_len'] = os.path.join(AUG_INFO_DIR, 'babble_len')

In [51]:
if not os.path.exists(VAL_MFCC_O):
    os.mkdir(VAL_MFCC_O)
else:
    shutil.rmtree(VAL_MFCC_O)
    os.mkdir(VAL_MFCC_O)
    
if not os.path.exists(VAL_LOGMELFB_O):
    os.mkdir(VAL_LOGMELFB_O)
else:
    shutil.rmtree(VAL_LOGMELFB_O)
    os.mkdir(VAL_LOGMELFB_O)
    
if not os.path.exists(VAL_RAWWAV_O):
    os.mkdir(VAL_RAWWAV_O)
else:
    shutil.rmtree(VAL_RAWWAV_O)
    os.mkdir(VAL_RAWWAV_O)

In [52]:
config = {}

config['sr'] = 16000
config['repeats'] = 5
config['batch_size'] = 1

In [53]:
def iter_data_preload(dataset, i):
    dataset.get_random_list()
    for count, (rw, fb, mfcc, label) in enumerate(dataset):
        file_name = str(i)+str('_')+str(count)
        with open(os.path.join(VAL_RAWWAV_O, file_name), 'wb') as handle:
            pickle.dump((rw.astype(np.float16), label.astype(np.int16)), handle)

        with open(os.path.join(VAL_LOGMELFB_O, file_name), 'wb') as handle:
            pickle.dump((fb.astype(np.float16), label.astype(np.int16)), handle)

        with open(os.path.join(VAL_MFCC_O, file_name), 'wb') as handle:
            pickle.dump((mfcc.astype(np.float16), label.astype(np.int16)), handle)
        start_time = time.time()

dataset = ThreeTypes_IterableDataset(config, data_dir_dict, data_len_dict)
dataset.noise_data_preload()

processes = [Process(target = iter_data_preload, args = (dataset, i)) for i in range(1)]
[p.start() for p in processes]
joined = [p.join() for p in processes]

preloading music_dict
100
200
300
400
500
600
preloading noise_dict
100
200
300
400
500
600
700
800
900
preloading babble_dict
100
200
300
400


# Make validation index

In [55]:
val_mfcc_dir = os.path.join(INFO_DIR, 'val_mfcc.csv')
val_fb_dir = os.path.join(INFO_DIR, 'val_fb.csv')
val_rw_dir = os.path.join(INFO_DIR, 'val_raw.csv')

In [56]:
mfcc_len = glob.glob(VAL_MFCC_O+'/*')
fb_len = glob.glob(VAL_LOGMELFB_O+'/*')
raw_len = glob.glob(VAL_RAWWAV_O+'/*')

In [57]:
print('mfcc', len(mfcc_len))
print('fb', len(fb_len))
print('raw', len(raw_len))

mfcc 1000
fb 1000
raw 1000


In [58]:
count = 0
with open(val_mfcc_dir, 'w') as f:
    for path in mfcc_len:
        assert os.path.isfile(path)
        f.write(path+'\n')
        count += 1
print('mfcc', count)

count = 0
with open(val_fb_dir, 'w') as f:
    for path in fb_len:
        assert os.path.isfile(path)
        f.write(path+'\n')
        count += 1
print('fb', count)

count = 0
with open(val_rw_dir, 'w') as f:
    for path in raw_len:
        assert os.path.isfile(path)
        f.write(path+'\n')
        count += 1
print('raw', count)

mfcc 1000
fb 1000
raw 1000
